In [1]:
import pandas as pd
import numpy as np
import folium
import requests

from bs4 import BeautifulSoup

# pip install tqdm : progressBar 구현
from tqdm.notebook import tqdm 
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# jupyter nbconvert --to script coffeeStore.ipynb

In [2]:
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [3]:
driver.maximize_window() # 윈도우 창 최대화

In [4]:
# 스타벅스) 매장 찾기-지역 검색 링크 주소
starbucks_url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(starbucks_url) # 해당 페이지로 이동하기

In [5]:
# 스타벅스) '서울' 링크 클릭
starbucks_seoul_selector = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_selector))).click()

In [6]:
# 스타벅스) '서울'-'전체' 클릭
starbuck_seoul_all = '#mCSB_2_container > ul > li:nth-child(1) > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbuck_seoul_all))).click()

In [7]:
# 스타벅스 html 코드를 파싱하여 html 파일에 기록합니다.
html = driver.page_source # 해당 페이지의 소스 코드 반환
filename = 'starbucks.html'
htmlfile = open(filename, 'w', encoding='UTF-8')
print(html, file=htmlfile)
htmlfile.close()
print(filename + ' 파일 생성됨')

starbucks.html 파일 생성됨


In [8]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# 파싱된 결과를 Beautiful Soup 객체로 생성합니다.
soup = BeautifulSoup(html, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [9]:
# 매장 정보들을 담고 있는 컨테이너 박스를 찾습니다.
# 컨테이너는 div 태그이고, id가 mCSB_3_container입니다.
container = soup.find('div', id='mCSB_3_container')
storeAll = container.find_all('li')
print('storeAll : %d' % len(storeAll))

storeAll : 615


In [19]:
starbucksData = []

for store in storeAll:
    # print(store)
    brand = '스타벅스'
    name = store.find('strong').text.strip()
    address = store.find('p').text.strip().replace('1522-3232', '')
    imsi = address.split(' ')
    gu = imsi[1]
    latitude = store['data-lat'] # 위도
    longitude = store['data-long'] # 경도

    starbucksData.append([brand, name, address, gu, latitude, longitude])
    # break

print(len(starbucksData))

615


In [ ]:
'''
<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>
'''